<a href="https://colab.research.google.com/github/Nikil263/llm-codechallenge/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Inference**

run pip install transformers if not already installed

In [4]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
import json

# Load the fine-tuned model and tokenizer
model_name = "Nikil263/Fine_Tuned_GPT2model"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Initialize the pipeline with the fine-tuned model
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [1]:
pyomo_template = """
from pyomo.environ import *

model = ConcreteModel()
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)
model.obj = Objective(expr=2 * model.x + 3 * model.y, sense=maximize)

# Define constraints
model.constraint1 = Constraint(expr=model.x + 2 * model.y <= 8)
model.constraint2 = Constraint(expr=3 * model.x + 2 * model.y <= 12)
{new_constraint}


# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)
"""

In [2]:
def add_constraint_to_model(user_input):
    # Generate the new constraint using the fine-tuned model
    response = generator(user_input, max_length=50, num_return_sequences=1)
    new_constraint_code = response[0]['generated_text'].strip()
    lines = new_constraint_code.split('\n')
    constraint_line = None

    for line in lines:
      if line.startswith("output: model.constraint3"):
        constraint_line = line
        break
    constraint_line = constraint_line.replace("output: ", "")
    # Inject the new constraint into the Pyomo template
    updated_model_code = pyomo_template.format(new_constraint=constraint_line)

    return updated_model_code

**Change input here**

In [5]:
user_request = "add a constraint of -3X + 7Y = 4 in our model." #change request here
updated_model_code = add_constraint_to_model(user_request)
print(updated_model_code)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



from pyomo.environ import *

model = ConcreteModel()
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)
model.obj = Objective(expr=2 * model.x + 3 * model.y, sense=maximize)

# Define constraints
model.constraint1 = Constraint(expr=model.x + 2 * model.y <= 8)
model.constraint2 = Constraint(expr=3 * model.x + 2 * model.y <= 12)
model.constraint3 = Constraint(expr=-3 * model.x + 7 * model.y = 4)


# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)

